In [1]:
from ArticlesRep import MeanSimilarityoneindustry,MeanSimilaritytwoindustry #common function

import pandas as pd

import numpy as np

%matplotlib inline

import pickle

import visdom

from tqdm import tnrange, tqdm_notebook
from tqdm.autonotebook import tqdm
tqdm.pandas()

from time import time,sleep
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
# import visdom
# vis=visdom.Visdom()
# env="TagBased"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

import re

from pprint import pprint
#coding:utf-8
import matplotlib.pyplot as plt

#有中文出现的情况，需要u'内容'
list_industry=["水泥","食品飲料","石化","紡織","電機機械","電器電纜","化學工業",
               "建材居家用品","造紙","鋼鐵金屬","車輛相關","科技相關","營建地產","運輸","觀光休閒娛樂",
               "金融相關","百貨通路","公用事業","控股","生技醫療保健","農林漁牧","航天軍工","能源","傳播出版","綜合",
               "傳產其他","其他","金屬礦採選",
              ]

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


## Data Set

In [5]:
picksample=np.load("D:3.AutoencoderForArticle/picksample.npy")
len(picksample)

12000

In [4]:
bow=np.load("D:3.AutoencoderForArticle/BOW_binary_v01.npy")

In [5]:
with open("D:3.AutoencoderForArticle/dict_collect_industry.p","rb") as f:
    dict_collect_industry=pickle.load(f)

## Reconstruct Model

In [6]:
from keras.models import load_model

In [7]:
#Autoencoder
autoencoder_untrained=load_model("autoencoder.initial.h5")
autoencoder_trained=load_model("best_weights.hdf5")

In [8]:
#Encoder_untrained
##layer
layer_encode=autoencoder_untrained.layers[1]
##dataflow
x=Input((20000,))
y=layer_encode(x)
##build Model
Encoder_untrained = Model(inputs=x,outputs=y)

In [9]:
#Encoder_rained
##layer
layer_encode=autoencoder_trained.layers[1]
##dataflow
x=Input((20000,))
y=layer_encode(x)
##build Model
Encoder_trained = Model(inputs=x,outputs=y)

## mean parewise Similarity in same industry

In [10]:
##除了同樣industry 也要比不同 industry 之間的

In [11]:
dict_=dict()
for k in dict_collect_industry:
    dict_[k]=len(dict_collect_industry[k])
df=pd.Series(dict_)
x=df.sort_values(ascending=False)
x

科技相關      6499
金融相關      1224
鋼鐵金屬      1030
車輛相關       392
生技醫療保健     341
營建地產       322
百貨通路       301
傳播出版       288
電機機械       214
能源         204
傳產其他       172
觀光休閒娛樂     166
紡織         145
運輸         117
化學工業        88
石化          86
公用事業        80
食品飲料        70
農林漁牧        66
造紙          55
水泥          49
建材居家用品      43
電器電纜        24
航天軍工        18
綜合           6
金屬礦採選        0
控股           0
其他           0
dtype: int64

In [ ]:
import 

### Similarity  In terms of Bow

In [12]:
dict_=dict()
for k in tqdm_notebook(dict_collect_industry):
    array=bow[dict_collect_industry[k]]
    if array.shape[0]!=0:
        dict_[k]=MeanSimilarityoneindustry(array)
y=pd.Series(dict_).sort_values(ascending=False)

In [13]:
pd.DataFrame(data=[y,x]).T.rename(columns={0:"similarity",1:"counts"})

,similarity,counts
傳播出版,0.678109,288.0
綜合,0.164056,6.0
建材居家用品,0.147406,43.0
水泥,0.140254,49.0
造紙,0.127686,55.0
能源,0.124400,204.0
紡織,0.119469,145.0
石化,0.119360,86.0
化學工業,0.109534,88.0
食品飲料,0.107475,70.0


### Similarity  In terms of untrained AE

In [26]:
dict_=dict()
for k in tqdm_notebook(dict_collect_industry):
    array=bow[dict_collect_industry[k]]
    if array.shape[0]!=0:
        array=Encoder_untrained.predict(array)
        dict_[k]=MeanSimilarityoneindustry(array)
y=pd.Series(dict_).sort_values(ascending=False)

In [28]:
pd.DataFrame(data=[y,x]).T.rename(columns={0:"similarity",1:"counts"})

,similarity,counts
傳播出版,0.776637,288.0
綜合,0.416461,6.0
建材居家用品,0.405265,43.0
水泥,0.398358,49.0
能源,0.389825,204.0
紡織,0.388938,145.0
石化,0.387904,86.0
造紙,0.385356,55.0
化學工業,0.380955,88.0
農林漁牧,0.379108,66.0


### Similarity  In terms of  AE

In [29]:
dict_=dict()
for k in tqdm_notebook(dict_collect_industry):
    array=bow[dict_collect_industry[k]]
    if array.shape[0]!=0:
        array=Encoder_trained.predict(array)
        dict_[k]=MeanSimilarityoneindustry(array)
y=pd.Series(dict_).sort_values(ascending=False)

In [30]:
pd.DataFrame(data=[y,x]).T.rename(columns={0:"similarity",1:"counts"})

,similarity,counts
傳播出版,0.969495,288.0
綜合,0.944694,6.0
能源,0.934721,204.0
建材居家用品,0.932537,43.0
水泥,0.920405,49.0
紡織,0.919983,145.0
公用事業,0.916855,80.0
石化,0.916222,86.0
農林漁牧,0.914296,66.0
觀光休閒娛樂,0.914113,166.0


In [24]:
Encoder_untrained.predict(np.arange(20000*2).reshape((2,20000))).shape

(2, 500)

## mean parewise Similarity in different industry

### Similarity  In terms of  Bow

In [58]:
array1=bow[dict_collect_industry["食品飲料"]]
array2=bow[dict_collect_industry["造紙"]]
MeanSimilaritytwoindustry(array1,array2)

0.09671060517993393

In [59]:
li=[]
for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
#         array1=Encoder_trained.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
#                     array2=Encoder_trained.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

百貨通路 百貨通路 0.08168207798577211
百貨通路 食品飲料 0.07361999178290989
百貨通路 生技醫療保健 0.06552719205466366
百貨通路 電機機械 0.0707188764199001
百貨通路 能源 0.07008906339128972
百貨通路 石化 0.06707286564641976
百貨通路 電器電纜 0.06657066816929681
百貨通路 觀光休閒娛樂 0.06915496140037407
百貨通路 科技相關 0.06493391897266221
百貨通路 營建地產 0.06581206164863997
百貨通路 水泥 0.07355201859190401
百貨通路 車輛相關 0.06575856358099548
百貨通路 運輸 0.0650521581544052
百貨通路 造紙 0.07275930287262568
百貨通路 航天軍工 0.06676315607510257
百貨通路 農林漁牧 0.07349177510729758
百貨通路 公用事業 0.06731268407613301
百貨通路 金融相關 0.0661646007342014
百貨通路 鋼鐵金屬 0.07131798329292756
百貨通路 傳產其他 0.07262246330244052
百貨通路 化學工業 0.07107842792338159
百貨通路 傳播出版 0.022973599013102752
百貨通路 綜合 0.07838986651699086
百貨通路 建材居家用品 0.07726085370270613
百貨通路 紡織 0.07097104598607684
食品飲料 百貨通路 0.07361999178290989
食品飲料 食品飲料 0.12022503982091236
食品飲料 生技醫療保健 0.08507824229844187
食品飲料 電機機械 0.08941792238282198
食品飲料 能源 0.07367406867603542
食品飲料 石化 0.08736170950325446
食品飲料 電器電纜 0.08809628372058854
食品飲料 觀光休閒娛樂 0.08596811996276088
食品飲料 科技相關 0.08486101

綜合 科技相關 0.06938863034028114
綜合 營建地產 0.07626451594466725
綜合 水泥 0.080228468541794
綜合 車輛相關 0.06678311178478466
綜合 運輸 0.07134684862106086
綜合 造紙 0.08457978969651292
綜合 航天軍工 0.06986832275886085
綜合 農林漁牧 0.07615004066644729
綜合 公用事業 0.07047864243682862
綜合 金融相關 0.07171263589577732
綜合 鋼鐵金屬 0.07674854014318593
綜合 傳產其他 0.08493912752978368
綜合 化學工業 0.07427991568718115
綜合 傳播出版 0.026482442523716426
綜合 綜合 0.30337960665534097
綜合 建材居家用品 0.08509573130978873
綜合 紡織 0.07639366497022615
建材居家用品 百貨通路 0.07726085370270613
建材居家用品 食品飲料 0.10853703791766925
建材居家用品 生技醫療保健 0.09402501370987261
建材居家用品 電機機械 0.1108014651692861
建材居家用品 能源 0.06767034871929738
建材居家用品 石化 0.09711228356126597
建材居家用品 電器電纜 0.10160532042059729
建材居家用品 觀光休閒娛樂 0.10106891903383375
建材居家用品 科技相關 0.10180690827012093
建材居家用品 營建地產 0.09563416495912305
建材居家用品 水泥 0.11859624792004617
建材居家用品 車輛相關 0.09198251623530192
建材居家用品 運輸 0.09369267059762988
建材居家用品 造紙 0.11272385675650537
建材居家用品 航天軍工 0.08318549433844699
建材居家用品 農林漁牧 0.06999663923072695
建材居家用品 公用事業 0.06754651124557

In [60]:
df=pd.DataFrame(li)
df=df.pivot(index=0,columns=1,values=2)

In [61]:
df

1,傳播出版,傳產其他,公用事業,化學工業,建材居家用品,水泥,營建地產,生技醫療保健,百貨通路,石化,...,觀光休閒娛樂,車輛相關,農林漁牧,造紙,運輸,金融相關,鋼鐵金屬,電器電纜,電機機械,食品飲料
0,,,,,,,,,,,,,,,,,,,,,
傳播出版,0.679227,0.021397,0.020225,0.020294,0.012639,0.016606,0.016476,0.019177,0.022974,0.034063,...,0.015048,0.016016,0.029422,0.026715,0.020923,0.024126,0.027530,0.013143,0.022725,0.016921
傳產其他,0.021397,0.091011,0.067001,0.083191,0.096556,0.085774,0.074029,0.075114,0.072622,0.075933,...,0.077714,0.073064,0.072140,0.085518,0.074428,0.066165,0.072635,0.077087,0.083380,0.083816
公用事業,0.020225,0.067001,0.091245,0.066011,0.067547,0.067952,0.061544,0.059890,0.067313,0.066932,...,0.059772,0.059879,0.077661,0.069633,0.059539,0.061994,0.071868,0.057013,0.064167,0.069868
化學工業,0.020294,0.083191,0.066011,0.119653,0.115056,0.103477,0.080038,0.081412,0.071078,0.095427,...,0.083713,0.081015,0.070519,0.102830,0.082966,0.064748,0.082036,0.086884,0.095416,0.097460
建材居家用品,0.012639,0.096556,0.067547,0.115056,0.167233,0.118596,0.095634,0.094025,0.077261,0.097112,...,0.101069,0.091983,0.069997,0.112724,0.093693,0.069834,0.084711,0.101605,0.110801,0.108537
水泥,0.016606,0.085774,0.067952,0.103477,0.118596,0.157800,0.093377,0.084168,0.073552,0.096043,...,0.087966,0.083656,0.070276,0.105507,0.088119,0.069485,0.085898,0.088843,0.093539,0.098819
營建地產,0.016476,0.074029,0.061544,0.080038,0.095634,0.093377,0.102937,0.071531,0.065812,0.073318,...,0.075086,0.067753,0.064382,0.082121,0.072942,0.064897,0.070343,0.072688,0.076119,0.079077
生技醫療保健,0.019177,0.075114,0.059890,0.081412,0.094025,0.084168,0.071531,0.103231,0.065527,0.072937,...,0.078013,0.070785,0.059618,0.079968,0.075115,0.063898,0.066219,0.076861,0.079559,0.085078
百貨通路,0.022974,0.072622,0.067313,0.071078,0.077261,0.073552,0.065812,0.065527,0.081682,0.067073,...,0.069155,0.065759,0.073492,0.072759,0.065052,0.066165,0.071318,0.066571,0.070719,0.073620


### Similarity  In terms of untrained AE

In [62]:
array1=Encoder_untrained.predict(bow[dict_collect_industry["食品飲料"]])
array2=Encoder_untrained.predict(bow[dict_collect_industry["造紙"]])
MeanSimilaritytwoindustry(array1,array2)

0.37503737

In [63]:
li=[]
for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=Encoder_untrained.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=Encoder_untrained.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

百貨通路 百貨通路 0.36709908
百貨通路 食品飲料 0.36200002
百貨通路 生技醫療保健 0.35520548
百貨通路 電機機械 0.36001703
百貨通路 能源 0.36006722
百貨通路 石化 0.35717502
百貨通路 電器電纜 0.35817397
百貨通路 觀光休閒娛樂 0.3574258
百貨通路 科技相關 0.35795918
百貨通路 營建地產 0.35532176
百貨通路 水泥 0.3612008
百貨通路 車輛相關 0.35755906
百貨通路 運輸 0.35689825
百貨通路 造紙 0.35802495
百貨通路 航天軍工 0.35323352
百貨通路 農林漁牧 0.36549887
百貨通路 公用事業 0.3596318
百貨通路 金融相關 0.3573209
百貨通路 鋼鐵金屬 0.36003077
百貨通路 傳產其他 0.36176664
百貨通路 化學工業 0.35938552
百貨通路 傳播出版 0.33297694
百貨通路 綜合 0.3621036
百貨通路 建材居家用品 0.36289072
百貨通路 紡織 0.35955945
食品飲料 百貨通路 0.36200002
食品飲料 食品飲料 0.38788962
食品飲料 生技醫療保健 0.36618403
食品飲料 電機機械 0.37039098
食品飲料 能源 0.36086413
食品飲料 石化 0.37074122
食品飲料 電器電纜 0.3676489
食品飲料 觀光休閒娛樂 0.36412373
食品飲料 科技相關 0.36968184
食品飲料 營建地產 0.36323485
食品飲料 水泥 0.37560445
食品飲料 車輛相關 0.36510825
食品飲料 運輸 0.36840004
食品飲料 造紙 0.37503737
食品飲料 航天軍工 0.35622177
食品飲料 農林漁牧 0.36500275
食品飲料 公用事業 0.36070725
食品飲料 金融相關 0.35826638
食品飲料 鋼鐵金屬 0.36500117
食品飲料 傳產其他 0.3691171
食品飲料 化學工業 0.37410408
食品飲料 傳播出版 0.3237731
食品飲料 綜合 0.36616498
食品飲料 建材居家用品 0.38

In [69]:
df_untrainAE=pd.DataFrame(li)
df_untrainAE=df_untrainAE.pivot(index=0,columns=1,values=2)

In [70]:
df_untrainAE

1,傳播出版,傳產其他,公用事業,化學工業,建材居家用品,水泥,營建地產,生技醫療保健,百貨通路,石化,...,觀光休閒娛樂,車輛相關,農林漁牧,造紙,運輸,金融相關,鋼鐵金屬,電器電纜,電機機械,食品飲料
0,,,,,,,,,,,,,,,,,,,,,
傳播出版,0.777412,0.332320,0.338405,0.326843,0.323095,0.322318,0.320636,0.323983,0.332977,0.322584,...,0.322816,0.326622,0.347439,0.310675,0.326657,0.329622,0.331867,0.328440,0.328612,0.323773
傳產其他,0.332320,0.375608,0.358393,0.368808,0.377082,0.369467,0.361349,0.362674,0.361767,0.364458,...,0.363971,0.363529,0.363260,0.367963,0.364676,0.357987,0.360348,0.364949,0.368181,0.369117
公用事業,0.338405,0.358393,0.373264,0.356438,0.357802,0.359423,0.354262,0.352251,0.359632,0.358653,...,0.353220,0.354330,0.368094,0.357065,0.354660,0.356160,0.360524,0.351580,0.356432,0.360707
化學工業,0.326843,0.368808,0.356438,0.387989,0.386149,0.378159,0.364321,0.365322,0.359386,0.372884,...,0.363765,0.368082,0.362393,0.375336,0.369256,0.355989,0.365787,0.367851,0.373827,0.374104
建材居家用品,0.323095,0.377082,0.357802,0.386149,0.419096,0.388831,0.372101,0.371703,0.362891,0.376699,...,0.372222,0.374801,0.362395,0.383803,0.376958,0.358422,0.367488,0.377563,0.383513,0.381487
水泥,0.322318,0.369467,0.359423,0.378159,0.388831,0.410637,0.372123,0.366276,0.361201,0.372656,...,0.367052,0.369571,0.364968,0.375935,0.372006,0.359393,0.367849,0.369132,0.371757,0.375604
營建地產,0.320636,0.361349,0.354262,0.364321,0.372101,0.372123,0.374373,0.357021,0.355322,0.360045,...,0.358842,0.358169,0.356556,0.364094,0.360709,0.353333,0.357076,0.359024,0.360589,0.363235
生技醫療保健,0.323983,0.362674,0.352251,0.365323,0.371703,0.366276,0.357021,0.377109,0.355206,0.360186,...,0.358560,0.359776,0.355483,0.363495,0.362513,0.355441,0.355735,0.361007,0.362709,0.366184
百貨通路,0.332977,0.361767,0.359632,0.359386,0.362891,0.361201,0.355322,0.355205,0.367099,0.357175,...,0.357426,0.357559,0.365499,0.358025,0.356898,0.357321,0.360031,0.358174,0.360017,0.362000


### Similarity  In terms of  AE

In [71]:
array1=Encoder_trained.predict(bow[dict_collect_industry["食品飲料"]])
array2=Encoder_trained.predict(bow[dict_collect_industry["造紙"]])
MeanSimilaritytwoindustry(array1,array2)

0.87964094

In [72]:
li=[]
for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=Encoder_trained.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=Encoder_trained.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

百貨通路 百貨通路 0.89698464
百貨通路 食品飲料 0.8734781
百貨通路 生技醫療保健 0.859852
百貨通路 電機機械 0.86773133
百貨通路 能源 0.9023963
百貨通路 石化 0.8851992
百貨通路 電器電纜 0.8750697
百貨通路 觀光休閒娛樂 0.87992376
百貨通路 科技相關 0.8555482
百貨通路 營建地產 0.87594736
百貨通路 水泥 0.87192553
百貨通路 車輛相關 0.8851425
百貨通路 運輸 0.8835829
百貨通路 造紙 0.8542819
百貨通路 航天軍工 0.87177265
百貨通路 農林漁牧 0.8979537
百貨通路 公用事業 0.90476555
百貨通路 金融相關 0.8955062
百貨通路 鋼鐵金屬 0.89149153
百貨通路 傳產其他 0.8773484
百貨通路 化學工業 0.87248677
百貨通路 傳播出版 0.87299085
百貨通路 綜合 0.9115508
百貨通路 建材居家用品 0.8452355
百貨通路 紡織 0.86178166
食品飲料 百貨通路 0.8734781
食品飲料 食品飲料 0.8954715
食品飲料 生技醫療保健 0.88416415
食品飲料 電機機械 0.88810223
食品飲料 能源 0.8645381
食品飲料 石化 0.89709
食品飲料 電器電纜 0.8986969
食品飲料 觀光休閒娛樂 0.90044945
食品飲料 科技相關 0.88165164
食品飲料 營建地產 0.8861082
食品飲料 水泥 0.9015812
食品飲料 車輛相關 0.8932834
食品飲料 運輸 0.8982115
食品飲料 造紙 0.87964094
食品飲料 航天軍工 0.86806166
食品飲料 農林漁牧 0.86106586
食品飲料 公用事業 0.87627894
食品飲料 金融相關 0.87084764
食品飲料 鋼鐵金屬 0.873117
食品飲料 傳產其他 0.88501096
食品飲料 化學工業 0.89748114
食品飲料 傳播出版 0.84036785
食品飲料 綜合 0.89549035
食品飲料 建材居家用品 0.89762545
食品飲料 紡織 0.900

In [75]:
df_trainAE=pd.DataFrame(li)
df_trainAE=df_trainAE.pivot(index=0,columns=1,values=2)

In [76]:
df_trainAE

1,傳播出版,傳產其他,公用事業,化學工業,建材居家用品,水泥,營建地產,生技醫療保健,百貨通路,石化,...,觀光休閒娛樂,車輛相關,農林漁牧,造紙,運輸,金融相關,鋼鐵金屬,電器電纜,電機機械,食品飲料
0,,,,,,,,,,,,,,,,,,,,,
傳播出版,0.969600,0.848700,0.894808,0.851221,0.796110,0.824538,0.833085,0.825963,0.872991,0.868723,...,0.855276,0.878318,0.879920,0.820905,0.855735,0.869278,0.867145,0.867777,0.850516,0.840368
傳產其他,0.848700,0.881584,0.881094,0.887573,0.881435,0.889970,0.882020,0.873754,0.877348,0.889839,...,0.893134,0.888228,0.867803,0.869348,0.891382,0.874789,0.873951,0.889882,0.879792,0.885011
公用事業,0.894808,0.881094,0.917894,0.877873,0.842243,0.872042,0.876334,0.856848,0.904766,0.894375,...,0.880385,0.892110,0.912113,0.858680,0.885814,0.902077,0.902851,0.878552,0.872616,0.876279
化學工業,0.851221,0.887573,0.877873,0.907443,0.907128,0.907331,0.888973,0.877469,0.872487,0.907133,...,0.900961,0.898024,0.864664,0.890211,0.900805,0.868040,0.878566,0.902516,0.896054,0.897481
建材居家用品,0.796110,0.881435,0.842243,0.907128,0.934106,0.917243,0.891242,0.884603,0.845236,0.894975,...,0.903789,0.884911,0.823476,0.891652,0.898570,0.841554,0.850571,0.900948,0.892636,0.897625
水泥,0.824538,0.889970,0.872042,0.907331,0.917243,0.922030,0.903400,0.887368,0.871926,0.905394,...,0.906729,0.893339,0.856308,0.893616,0.907712,0.871791,0.876890,0.901489,0.893343,0.901581
營建地產,0.833085,0.882021,0.876335,0.888973,0.891242,0.903400,0.899939,0.876534,0.875947,0.889941,...,0.895852,0.881934,0.861961,0.874387,0.898000,0.879141,0.874944,0.886649,0.876657,0.886108
生技醫療保健,0.825963,0.873754,0.856848,0.877469,0.884603,0.887368,0.876534,0.896375,0.859852,0.872263,...,0.898545,0.880244,0.832998,0.855801,0.889854,0.860977,0.847014,0.891866,0.871994,0.884164
百貨通路,0.872991,0.877348,0.904766,0.872487,0.845236,0.871926,0.875947,0.859852,0.896985,0.885199,...,0.879924,0.885143,0.897954,0.854282,0.883583,0.895506,0.891492,0.875070,0.867731,0.873478


In [ ]:
# Metrics mean sum similarity one to multi

In [130]:
#bow
li=[]
for i,S in df[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.679227,0.021070,0.329078
12,綜合,0.303380,0.072577,0.115401
4,建材居家用品,0.167233,0.090794,0.038220
5,水泥,0.157800,0.084454,0.036673
13,能源,0.128692,0.066197,0.031247
18,造紙,0.143547,0.084130,0.029708
22,電器電纜,0.130438,0.072582,0.028928
14,航天軍工,0.125201,0.068092,0.028555
9,石化,0.129600,0.077346,0.026127
11,紡織,0.125542,0.083416,0.021063


In [132]:
#UAE
li=[]
for i,S in df_untrainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.777412,0.327992,0.224710
12,綜合,0.513718,0.361728,0.075995
4,建材居家用品,0.419096,0.371700,0.023698
5,水泥,0.410637,0.367451,0.021593
13,能源,0.392816,0.356130,0.018343
14,航天軍工,0.388210,0.353675,0.017268
9,石化,0.395021,0.362884,0.016069
22,電器電纜,0.392299,0.360175,0.016062
18,造紙,0.396531,0.364573,0.015979
17,農林漁牧,0.388515,0.361017,0.013749


In [133]:
#AE
li=[]
for i,S in df_trainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.969600,0.852941,0.058330
13,能源,0.935041,0.874088,0.030477
4,建材居家用品,0.934106,0.875658,0.029224
12,綜合,0.953911,0.896704,0.028604
17,農林漁牧,0.915595,0.870874,0.022360
11,紡織,0.920534,0.885247,0.017644
2,公用事業,0.917894,0.883162,0.017366
5,水泥,0.922030,0.887901,0.017064
7,生技醫療保健,0.896375,0.869223,0.013576
15,觀光休閒娛樂,0.914631,0.889304,0.012664


ImportError: No module named 'spherecluster'

### Similarity  In terms of  AE with supervision (Model one layer ,20000 common word)

In [5]:
Data=np.load("D:3.AutoencoderForArticle/BOW_binary_v01.npy")

In [6]:
with open("D:3.AutoencoderForArticle/train_dict_collect_small_industry","rb") as f:
    train_dict_collect_small_industry=pickle.load(f)

In [7]:
with open("D:3.AutoencoderForArticle/test_dict_collect_small_industry","rb") as f:
    test_dict_collect_small_industry=pickle.load(f)

In [8]:
from keras.models import load_model

Using TensorFlow backend.


In [9]:
# custommed function
def noisefunction(x):
    x_noise=K.ones_like(x)
    return K.in_train_phase(x_noise,x,1)

def tripletlossfunction(inputs):
    r01=inputs[0]
    r02=inputs[1]
    loss=K.log(1+K.exp(r02-r01))
#     x=np.array([[0],[1],[0],[1]])
#     x=K.variable(x)
    return loss

def losspassfunction(y_true,y_pred):
    return y_pred

def test(inputs):
#     x=K.dot(inputs,k.transpose(inputs))
    x=K.transpose(inputs)
    return x

In [11]:
Tri_AutoEncoder=load_model("Tri_AutoEncoder_trained.h5",custom_objects={"losspassfunction":losspassfunction})

In [12]:
encoder_in_AE=Tri_AutoEncoder.layers[3]

### on Trainset

In [14]:
li=[]
model=encoder_in_AE
dict_collect_industry=train_dict_collect_small_industry

for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=model.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=model.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

生技醫療保健 生技醫療保健 0.9503694
生技醫療保健 百貨通路 0.64897704
生技醫療保健 傳播出版 0.59648645
生技醫療保健 營建地產 0.60032886
生技醫療保健 車輛相關 0.6357869
百貨通路 生技醫療保健 0.648977
百貨通路 百貨通路 0.951421
百貨通路 傳播出版 0.62093025
百貨通路 營建地產 0.64138734
百貨通路 車輛相關 0.65031534
傳播出版 生技醫療保健 0.5964864
傳播出版 百貨通路 0.62093025
傳播出版 傳播出版 0.9747632
傳播出版 營建地產 0.6003573
傳播出版 車輛相關 0.61065286
營建地產 生技醫療保健 0.6003288
營建地產 百貨通路 0.64138734
營建地產 傳播出版 0.6003573
營建地產 營建地產 0.9443346
營建地產 車輛相關 0.61651313
車輛相關 生技醫療保健 0.6357869
車輛相關 百貨通路 0.6503153
車輛相關 傳播出版 0.6106529
車輛相關 營建地產 0.61651313
車輛相關 車輛相關 0.96131873



In [16]:
df_trainAE=pd.DataFrame(li)
df_trainAE=df_trainAE.pivot(index=0,columns=1,values=2)

In [17]:
df_trainAE

1,傳播出版,營建地產,生技醫療保健,百貨通路,車輛相關
0,,,,,
傳播出版,0.974763,0.600357,0.596486,0.620930,0.610653
營建地產,0.600357,0.944335,0.600329,0.641387,0.616513
生技醫療保健,0.596486,0.600329,0.950369,0.648977,0.635787
百貨通路,0.620930,0.641387,0.648977,0.951421,0.650315
車輛相關,0.610653,0.616513,0.635787,0.650315,0.961319


In [18]:
#AE
li=[]
for i,S in df_trainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.974763,0.607107,0.183828
4,車輛相關,0.961319,0.628317,0.166501
2,生技醫療保健,0.950369,0.620395,0.164987
1,營建地產,0.944335,0.614647,0.164844
3,百貨通路,0.951421,0.640402,0.155509


#### on testset

In [21]:
li=[]
model=encoder_in_AE
dict_collect_industry=test_dict_collect_small_industry

for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=model.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=model.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

生技醫療保健 生技醫療保健 0.932781
生技醫療保健 百貨通路 0.7199085
生技醫療保健 傳播出版 0.62326825
生技醫療保健 營建地產 0.64538056
生技醫療保健 車輛相關 0.7225844
百貨通路 生技醫療保健 0.7199085
百貨通路 百貨通路 0.9336065
百貨通路 傳播出版 0.6854175
百貨通路 營建地產 0.72235763
百貨通路 車輛相關 0.7555785
傳播出版 生技醫療保健 0.6232683
傳播出版 百貨通路 0.6854175
傳播出版 傳播出版 0.96257
傳播出版 營建地產 0.62382054
傳播出版 車輛相關 0.6517313
營建地產 生技醫療保健 0.64538056
營建地產 百貨通路 0.72235763
營建地產 傳播出版 0.6238205
營建地產 營建地產 0.92721236
營建地產 車輛相關 0.6822071
車輛相關 生技醫療保健 0.7225844
車輛相關 百貨通路 0.7555786
車輛相關 傳播出版 0.65173125
車輛相關 營建地產 0.68220717
車輛相關 車輛相關 0.9331247



In [22]:
df_trainAE=pd.DataFrame(li)
df_trainAE=df_trainAE.pivot(index=0,columns=1,values=2)

In [23]:
df_trainAE

1,傳播出版,營建地產,生技醫療保健,百貨通路,車輛相關
0,,,,,
傳播出版,0.962570,0.623821,0.623268,0.685417,0.651731
營建地產,0.623820,0.927212,0.645381,0.722358,0.682207
生技醫療保健,0.623268,0.645381,0.932781,0.719908,0.722584
百貨通路,0.685417,0.722358,0.719908,0.933607,0.755579
車輛相關,0.651731,0.682207,0.722584,0.755579,0.933125


In [25]:
#AE
li=[]
for i,S in df_trainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.962570,0.646059,0.158255
1,營建地產,0.927212,0.668441,0.129385
2,生技醫療保健,0.932781,0.677785,0.127498
4,車輛相關,0.933125,0.703025,0.115050
3,百貨通路,0.933607,0.720816,0.106395


### Similarity  In terms of  AE with supervision (Model 2 layer ,common 19404 words)

In [2]:
Data=np.load("D:3.AutoencoderForArticle/BOW_binary_v02.npy")

In [7]:
with open("D:3.AutoencoderForArticle/train_dict_collect_small_industry","rb") as f:
    train_dict_collect_small_industry=pickle.load(f)

In [8]:
with open("D:3.AutoencoderForArticle/test_dict_collect_small_industry","rb") as f:
    test_dict_collect_small_industry=pickle.load(f)

In [9]:
from keras.models import load_model

In [10]:
# custommed function
def noisefunction(x):
    x_noise=K.ones_like(x)
    return K.in_train_phase(x_noise,x,1)

def tripletlossfunction(inputs):
    r01=inputs[0]
    r02=inputs[1]
    loss=K.log(1+K.exp(r02-r01))
#     x=np.array([[0],[1],[0],[1]])
#     x=K.variable(x)
    return loss

def losspassfunction(y_true,y_pred):
    return y_pred

def test(inputs):
#     x=K.dot(inputs,k.transpose(inputs))
    x=K.transpose(inputs)
    return x

In [11]:
Tri_AutoEncoder=load_model("Models/Model2_addlayer/bestmodel.hdf5",custom_objects={"losspassfunction":losspassfunction})

In [12]:
encoder_in_AE=Tri_AutoEncoder.layers[3]

### on Trainset

In [17]:
li=[]
model=encoder_in_AE
dict_collect_industry=train_dict_collect_small_industry
bow=Data
for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=model.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=model.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

車輛相關 車輛相關 0.99695426
車輛相關 營建地產 0.8590331
車輛相關 百貨通路 0.87698984
車輛相關 傳播出版 0.7820184
車輛相關 生技醫療保健 0.8691423
營建地產 車輛相關 0.8590331
營建地產 營建地產 0.997325
營建地產 百貨通路 0.78731096
營建地產 傳播出版 0.8653716
營建地產 生技醫療保健 0.8804796
百貨通路 車輛相關 0.87698996
百貨通路 營建地產 0.78731096
百貨通路 百貨通路 0.99797547
百貨通路 傳播出版 0.87209564
百貨通路 生技醫療保健 0.8546969
傳播出版 車輛相關 0.7820185
傳播出版 營建地產 0.86537164
傳播出版 百貨通路 0.87209564
傳播出版 傳播出版 0.9970335
傳播出版 生技醫療保健 0.8298781
生技醫療保健 車輛相關 0.8691423
生技醫療保健 營建地產 0.8804796
生技醫療保健 百貨通路 0.85469687
生技醫療保健 傳播出版 0.82987815
生技醫療保健 生技醫療保健 0.9975396



In [18]:
df_trainAE=pd.DataFrame(li)
df_trainAE=df_trainAE.pivot(index=0,columns=1,values=2)

In [19]:
df_trainAE

1,傳播出版,營建地產,生技醫療保健,百貨通路,車輛相關
0,,,,,
傳播出版,0.997033,0.865372,0.829878,0.872096,0.782018
營建地產,0.865372,0.997325,0.880480,0.787311,0.859033
生技醫療保健,0.829878,0.880480,0.997540,0.854697,0.869142
百貨通路,0.872096,0.787311,0.854697,0.997975,0.876990
車輛相關,0.782018,0.859033,0.869142,0.876990,0.996954


In [18]:
#AE
li=[]
for i,S in df_trainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.974763,0.607107,0.183828
4,車輛相關,0.961319,0.628317,0.166501
2,生技醫療保健,0.950369,0.620395,0.164987
1,營建地產,0.944335,0.614647,0.164844
3,百貨通路,0.951421,0.640402,0.155509


### on testset

In [13]:
li=[]
model=encoder_in_AE
dict_collect_industry=test_dict_collect_small_industry
bow=Data
for k in tqdm_notebook(dict_collect_industry):
    array1=bow[dict_collect_industry[k]]
    if len(array1)!=0:
        array1=model.predict(array1)
        for j in dict_collect_industry:
                array2=bow[dict_collect_industry[j]]
                if len(array2)!=0:
                    array2=model.predict(array2)
                    simi=MeanSimilaritytwoindustry(array1,array2)
                    print(k,j,simi)
                    li.append((k,j,simi))

車輛相關 車輛相關 0.9832656
車輛相關 營建地產 0.87183195
車輛相關 百貨通路 0.9007254
車輛相關 傳播出版 0.8057547
車輛相關 生技醫療保健 0.887121
營建地產 車輛相關 0.871832
營建地產 營建地產 0.9850005
營建地產 百貨通路 0.82354563
營建地產 傳播出版 0.87027884
營建地產 生技醫療保健 0.886237
百貨通路 車輛相關 0.9007254
百貨通路 營建地產 0.82354563
百貨通路 百貨通路 0.9874803
百貨通路 傳播出版 0.88492095
百貨通路 生技醫療保健 0.8794175
傳播出版 車輛相關 0.8057547
傳播出版 營建地產 0.87027884
傳播出版 百貨通路 0.88492095
傳播出版 傳播出版 0.9850525
傳播出版 生技醫療保健 0.8383763
生技醫療保健 車輛相關 0.88712096
生技醫療保健 營建地產 0.886237
生技醫療保健 百貨通路 0.87941754
生技醫療保健 傳播出版 0.83837634
生技醫療保健 生技醫療保健 0.98086125



In [14]:
df_trainAE=pd.DataFrame(li)
df_trainAE=df_trainAE.pivot(index=0,columns=1,values=2)

In [15]:
df_trainAE

1,傳播出版,營建地產,生技醫療保健,百貨通路,車輛相關
0,,,,,
傳播出版,0.985053,0.870279,0.838376,0.884921,0.805755
營建地產,0.870279,0.985000,0.886237,0.823546,0.871832
生技醫療保健,0.838376,0.886237,0.980861,0.879418,0.887121
百貨通路,0.884921,0.823546,0.879417,0.987480,0.900725
車輛相關,0.805755,0.871832,0.887121,0.900725,0.983266


In [16]:
#AE
li=[]
for i,S in df_trainAE[:].iterrows():
    R=S.loc[i]
    UR=S.drop(i).mean()
    result=(R-UR)/2
    li.append((i,R,UR,result))
pd.DataFrame(li).rename(columns={0:"industry",1:"R",2:"UR",3:"(R-UR)/2"}).sort_values("(R-UR)/2",ascending=False)

,industry,R,UR,(R-UR)/2
0,傳播出版,0.985053,0.849833,0.067610
1,營建地產,0.985000,0.862973,0.061014
4,車輛相關,0.983266,0.866358,0.058454
3,百貨通路,0.987480,0.872152,0.057664
2,生技醫療保健,0.980861,0.872788,0.054037
